In [1]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
url = 'https://www.saramin.co.kr/'
keyword = '딥러닝'

browser = webdriver.Chrome()
browser.get(url)

In [3]:
search_btn = browser.find_element(By.CLASS_NAME, 'btn_search').click()

keyword_input = browser.find_element(By.ID, 'ipt_keyword_recruit')
keyword_input.click()
keyword_input.send_keys(keyword)

btn_search_recruit = browser.find_element(By.ID, 'btn_search_recruit')
btn_search_recruit.click()

In [4]:
total_data = []

idx = 1
while True:
    if idx == 6:
        break

    if idx > 1:
        url = f'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=search&searchword=%EB%94%A5%EB%9F%AC%EB%8B%9D&recruitPage={idx}&recruitSort=relation&recruitPageCount=40&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting='
        try:
            browser.get(url)
        except:
            print(f'Page Not Found. Current Page No : {idx}')
            break

    time.sleep(3)
    container = browser.find_element(By.CLASS_NAME, 'content')
    blocks = container.find_elements(By.CLASS_NAME, 'item_recruit')
    
    print(f"Page No : {idx}, There are {len(blocks)} job openings.", end=" ")
    for i, block in enumerate(blocks):
        try:
            company_name = block.find_element(By.CLASS_NAME, 'corp_name').find_element(By.TAG_NAME, 'a').text
            
            job_tit = block.find_element(By.CLASS_NAME, 'job_tit').find_element(By.TAG_NAME, 'a')
            title = job_tit.get_attribute('title')
            link = job_tit.get_attribute('href')
            
            options = []
            job_condition = block.find_element(By.CLASS_NAME, 'job_condition').find_elements(By.TAG_NAME, 'span')
            for j, condition in enumerate(job_condition):
                if j == 0:
                    regions = condition.find_elements(By.TAG_NAME, 'a')
                    regions = ','.join([region.text for region in regions])
                    options.append(regions)
                else:
                    options.append(condition.text)
            options = ','.join([option for option in options])

            detail_fields = block.find_element(By.CLASS_NAME, 'job_sector').find_elements(By.TAG_NAME, 'a')
            detail_fields = ','.join([x.text for x in detail_fields])

            data = {'회사명' : company_name, '채용공고 제목' : title, '채용공고 세부 사항' : options, '기술 세부 사항' : detail_fields, '링크' : link}
            total_data.append(data)
        
        except NoSuchElementException as e:
            print(f'\n\tBlock No : {i}, \n\t{e}')

    print(f"--> Current num of data : {len(total_data)}")
    idx += 1

Page No : 1, There are 40 job openings. --> Current num of data : 40
Page No : 2, There are 40 job openings. --> Current num of data : 80
Page No : 3, There are 40 job openings. --> Current num of data : 120
Page No : 4, There are 40 job openings. --> Current num of data : 160
Page No : 5, There are 40 job openings. --> Current num of data : 200


In [5]:
df = pd.DataFrame(total_data)
df.to_csv("Saramin.csv", encoding='utf-8-sig')
browser.close()